In [ ]:
# # import the modules 
import praw 
from datetime import datetime
from textblob import TextBlob 
import networkx as nx, numpy as np, pandas as pd, matplotlib.pyplot as plt , re, time
%matplotlib inline

: 

In [ ]:
# # creating an authorized reddit instance 
reddit = praw.Reddit(
    client_id='IZh-17KfLVsGl6boaOga-A',
    client_secret='1U1ejYRbJj6gac8Y2rIx6dW9N0XjJw',
    password='China@123',
    user_agent='ChardMany8896',
    username='ChardMany8896',
    )

: 

In [ ]:
def get_posts(subred_name, n):
    subreddit = reddit.subreddit(subred_name)
    posts_info = [] 
    
    for subm in subreddit.top(limit=n):
        
        subred_info = []
        subred_info.append(subm.id)  
        subred_info.append(str(subm.author)) 
        subred_info.append(subm.score)  
        subred_info.append(subm.upvote_ratio)
        subred_info.append(subm.num_comments)
        subred_info.append(subm.subreddit)
        posts_info.append(subred_info)
    
    sorted_info = sorted(posts_info, key=lambda x: x[1], reverse = True)
    posts_df = pd.DataFrame(sorted_info, columns = ['id','author', 'score','upvote_ratio' ,'num_comments', 'subreddit'])
    return posts_df
python_df = get_posts('Python', 100)
python_df.head(100)



: 

In [ ]:
# Data Cleaning

freq_authors = python_df[python_df.duplicated(['author'], keep = False)] # users who have posted more than once
freq_authors = freq_authors[freq_authors.author != 'None'] #  Getting rid of deleted users
freq_authors.author.nunique() #There are 41 users out of 500 who have posted more than once

# The bar graph shows the distribution of users out of 500 who featured posts on r/python, more than once.

plt.figure(figsize=(10, 5))
author_counts = freq_authors['author'].value_counts()
if not author_counts.empty:
    ax = author_counts.plot(kind='bar', title='Distribution of author/users and their posts')
    ax.set_xlabel("Users")
    ax.set_ylabel("Number of posts")
else:
    print("No data to plot")


: 

In [ ]:
authors_lst = ["Python",'AskReddit', 'Programming', 'machinelearning', "gaming", "worldnews",
             "todayilearned", "Music", "movies", "science", "pics", "memes", "Jokes",
             "InternetIsBeautiful", "technology"]  

authors_df =  pd.DataFrame() 

for u in authors_lst: 
    c = get_posts(u, 10)
    authors_df = pd.concat([authors_df, c])

authors_df.head(11)

: 

In [ ]:
# NETWORK ANALYSIS USING NetworkX
nx_df = authors_df[['author','subreddit',]]
nx_df.head()


: 

In [ ]:

# Graph with nodes('subreddits) and edges('authors/users')
g = nx.from_pandas_edgelist(nx_df, source='author', target='subreddit')
print(g)

: 

In [ ]:
# Top 10 Subreddits with maximum Degree Centrality
centrality = nx.degree_centrality(g)
[(x,centrality[x]) for x in sorted(centrality, key = centrality.get, reverse = True)[:10]]

: 

In [ ]:
# Top 10 Subreddits with maximum Closeness Centrality
centrality = nx.closeness_centrality(g)
[(x,centrality[x]) for x in sorted(centrality, key = centrality.get, reverse = True)[:10]]

: 

In [ ]:
# Betweenness Centrality
centrality = nx.betweenness_centrality(g)
[(x,centrality[x]) for x in sorted(centrality, key = centrality.get, reverse = True)[:10]]

: 

In [ ]:
# Eigenvector Centrality
centrality = nx.eigenvector_centrality(g)
[(x,centrality[x]) for x in sorted(centrality, key = centrality.get, reverse = True)[:10]]

: 

In [ ]:
# GRAPH VISUALIZATIONS
from matplotlib.pyplot import figure
figure(figsize=(10, 10))
nx.draw_shell(g, with_labels=True)

: 

In [ ]:
leaderboard = {}
for x in g.nodes:
 leaderboard[x] = len(g[x])
s = pd.Series(leaderboard, name='connections')
df_conn = s.to_frame().sort_values('connections', ascending=False)
df_conn.head()

: 

In [ ]:


def get_related_subreddits(subreddit_name, n):
    subreddit = reddit.subreddit(subreddit_name)
    related_subreddits = {}

    # Get n hot posts from the subreddit
    for submission in subreddit.hot(limit=n):
        # Get a list of subreddits mentioned in the comments
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            for word in comment.body.split():
                if word.lower().startswith('r/'):
                    related_subreddits[word.lower()] = related_subreddits.get(word.lower(), 0) + 1

    return related_subreddits

# Get related subreddits
related_subreddits = get_related_subreddits('Python', 500)

# Create a graph
G = nx.Graph()

# Add edges to the graph
for subreddit, weight in related_subreddits.items():
    G.add_edge('Python', subreddit, weight=weight)

# Draw the graph
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, seed=42)  # positions for all nodes
nx.draw(G, pos, with_labels=True)
plt.show()


: 

In [ ]:

# Assuming nx_df is a DataFrame with 'author' and 'subreddit' columns
g = nx.from_pandas_edgelist(nx_df, source='author', target='subreddit')

# Create a layout for nodes 
layout = nx.spring_layout(g, iterations=50, scale=2)

subs = list(nx_df['subreddit'].unique())


subs = [sub for sub in subs if sub in g.nodes]
authors_lst = [author for author in authors_lst if author in g.nodes]


nx.draw_networkx_nodes(g, 
                       layout, 
                       nodelist=subs, 
                       node_size=sub_size, 
                       node_color='powderblue')

# Assuming authors_lst is a list of authors
nx.draw_networkx_nodes(g, layout, nodelist=authors_lst, node_color='green', node_size=100)

# Draw highly connected influencers 
popular_people = [person for person in authors_lst if g.degree(person) > 1]
nx.draw_networkx_nodes(g, layout, nodelist=popular_people, node_color='orange', node_size=100)

nx.draw_networkx_edges(g, layout, width=1, edge_color="lightgreen")

node_labels = dict(zip(subs, subs)) #labels for subs
nx.draw_networkx_labels(g, layout, labels=node_labels)

plt.axis('off')
plt.title("Network Graph of Related Subreddits")
plt.show()


: 

: 